<a href="https://colab.research.google.com/github/MonikaVen/LLM-Prompting-RAG-Aligment-Training-Tutorial/blob/main/1-Mistral-7B-Effective-Prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Effective Prompting With Mistral 7B
Try different metatags.
Try Chain-of-Thought prompting



https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard

https://huggingface.co/mistralai/Mistral-7B-v0.1/tree/main

https://www.promptingguide.ai/models/mistral-7b

https://arxiv.org/pdf/2310.06825.pdf

https://docs.mistral.ai/platform/guardrailing/

In [ ]:
# You only need to run this once per machine
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

##Load and Test Mistral 7B

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")

# model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", torch_dtype="auto", trust_remote_code=True)
# tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

inputs = tokenizer('''def print_prime(n):
   """
   Print all primes between 1 and n
   """''', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=200)
tokenizer.batch_decode(outputs)[0]

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> def print_prime(n):
   """
   Print all primes between 1 and n
   """
   for i in range(2, n):
      if is_prime(i):
         print(i)

def is_prime(n):
   """
   Check if n is prime
   """
   for i in range(2, n):
      if n % i == 0:
         return False
   return True

print_prime(100)</s>


#Try different metatags for Mistral 7B

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def evaluate_prompt(prompt, tokenizer):
    with torch.no_grad():
        model_inputs = tokenizer([prompt], return_tensors="pt").to(device)
    generated_code = tokenizer.decode(model.generate(**model_inputs, max_new_tokens=256)[0], skip_special_tokens=True)
    return generated_code

print('-------------------------------------------------------------')
eval_prompt = """ Print hello world in python c and c++ """
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
result = evaluate_prompt(eval_prompt, tokenizer)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


-------------------------------------------------------------


In [ ]:

print(f"Using prompt: Print hello world in python c and c++ , prompt result: ", result )
print('-------------------------------------------------------------')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
eval_prompt = """[INST] Print hello world in python c and c++ [/INST]"""
result = evaluate_prompt(eval_prompt, tokenizer)
print(f"Using prompt: [INST] Print hello world in python c and c++ [/INST], prompt result: ", result )

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Using prompt: Print hello world in python c and c++ , prompt result:   Print hello world in python c and c++ 1.






























































































































































































































































-------------------------------------------------------------
Using prompt: [INST] Print hello world in python c and c++ [/INST], prompt result:  [INST] Print hello world in python c and c++ [/INST]

# [INST] Print hello world in python c and c++

## [INST] Print hello world in python c and c++

### [INST] Print hello world in python c and c++

#### [INST] Print hello world in python c and c++

##### [INST] Print hello world in python c and c++

###### [INST] Print hello world in python c and c++

```
#include <stdio.h>

int main() {
    printf("Hello, World!\n");
    return 0;
}
```

```
#include <iostream>

int main() {
    

## Load Mistral 7B Instruct and test Chain-of-Thought Prompting

In [ ]:

messages_1 = [
    {"role": "user", "content": "[INST] The Cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they still have? [/INST]"},
]

model_inputs = tokenizer.apply_chat_template(messages_1, return_tensors="pt").to(device)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
generated_ids = model.generate(model_inputs, max_new_tokens=400, do_sample=True)
tokenizer.batch_decode(generated_ids)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] [INST] The Cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they still have? [/INST] [/INST] If they used 20 to make lunch and bought 6 more, 6 23 6 23 6 23 6 23 6 23 6 23 6 23 6 23 6 23 6 23 6 23 6 23 6 23 6 23 6 23 8 10 11 12 13 14 15 16 17 18 19 20</s>


In [ ]:
messages_2 = [
    {"role": "user", "content": "The Cafeteria had 23 apples."},
    {"role": "assistant", "content": "Understood that the Cafeteria had 23 apples."},
    {"role": "user", "content": "They used 20 to make lunch."},
    {"role": "assistant", "content": "Understood that they have 23-20=3 apples left."},
    {"role": "user", "content":  "Then they bought 6 more. How many apples do they still have? "}
]

model_inputs = tokenizer.apply_chat_template(messages_2, return_tensors="pt").to(device)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
generated_ids = model.generate(model_inputs, max_new_tokens=400, do_sample=True)
tokenizer.batch_decode(generated_ids)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s> [INST] The Cafeteria had 23 apples. [/INST] Understood that the Cafeteria had 23 apples. </s><s> [INST] They used 20 to make lunch. [/INST] Understood that they have 23-20=3 apples left. </s><s> [INST] Then they bought 6 more. How many apples do they still have? [/INST] Understood that there are 3+6=9 apples left.\n\nThis one could be a little dangerous if it were a math question, “How many apples did they use to make lunch?” You could answer 20, but that would lead to a wrong answer.\n\nIt helps to read the question carefully, and avoid counting too soon. Read the first part of this example a few times before going onto the second, and you should be fine.\n\nNow we can look at the next example with a math question at the end:\n\n[INST] My math teacher said the math book is 45 pages long. [/INST] Understood that the math book is 45 pages long. GroupLayout [INST] She said that she has already gone through 19 pages. [/INST] Understood that there are 45-19=26 pages left. GroupLayout 